In [1]:
isColab = True
try:
  from oauth2client.client import GoogleCredentials
  from pydrive.drive import GoogleDrive
  from google.colab import auth, drive
  from pydrive.auth import GoogleAuth
except:
  isColab = False
from random import randint, shuffle, random
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
import numpy as np
import zipfile
import shutil
import glob
import cv2
import os

In [2]:
drive.mount('/content/drive', force_remount=True)
if isColab:
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  my_drive = GoogleDrive(gauth)
  if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
ROOT_DIR = '/content/drive/MyDrive/University/UniPI/Deep Learning/Project' if isColab else os.path.curdir

if isColab:
  zip_dir = os.path.join(ROOT_DIR, 'immagini.zip')
  if not os.path.exists('immagini'):
    with zipfile.ZipFile(zip_dir, 'r') as zip_ref:
        zip_ref.extractall('immagini')


In [4]:
class ImageLoader:
  def __init__(self, root='', batch_size=1):
    self.__dir = os.path.join(root, "immagini", f"DIV2K_valid")
    self.__images = os.listdir(os.path.join(self.__dir))
    shuffle(self.__images)
    self.__nimages = len(self.__images)
    self.__batch_size = batch_size
    self.__index = 0
    self.__len = self.__nimages // self.__batch_size + (1 if self.__nimages % self.__batch_size != 0 else 0)

  def reset(self):
    shuffle(self.__images)
    self.__index = 0
  def batch_size(self):
    return self.__batch_size
  def nimages(self):
    return self.__nimages

  def __len__(self):
    return self.__len
  def __iter__(self):
    return self

  def __get_item__(self, index):
    original_x1 = cv2.cvtColor(cv2.imread(os.path.join(self.__dir, self.__images[index])), cv2.COLOR_BGR2RGB)

    rh, rw = (randint(0, original_x1.shape[0]-256), randint(0, original_x1.shape[1]-256))
    original_x1 = original_x1[rh:rh+256, rw:rw+256]

    rotate, rotation_choice = (randint(0, 1) == 1, [cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_180, cv2.ROTATE_90_COUNTERCLOCKWISE][randint(0, 2)])
    flip, flip_choice = (randint(0, 1) == 1, randint(0, 1))

    if rotate:
      original_x1 = cv2.rotate(original_x1, rotation_choice)
    if flip:
      original_x1 = cv2.flip(original_x1, flip_choice)

    lowres = cv2.resize(original_x1, (64,64), interpolation=cv2.INTER_CUBIC)

    hr = np.array(original_x1, dtype=np.float32)/255.
    lr = np.array(lowres, dtype=np.float32)/255.
    return hr, lr

  def __next__(self):
    if self.__index == self.__len:
      self.reset()
      raise StopIteration
    batch_hr = []
    batch_lr = []
    for i in range(self.__batch_size):
      current_index = self.__index*self.__batch_size + i
      if current_index >= self.__nimages:
        break
      hr, lr = self.__get_item__(current_index)
      batch_hr.append(hr)
      batch_lr.append(lr)
    self.__index+=1
    return tf.stack(batch_hr), tf.stack(batch_lr)

In [5]:
MODEL_PATH = os.path.join(ROOT_DIR, "models", "generator.h5")
assert os.path.exists(MODEL_PATH), "model not found"
generator = None
with tf.keras.utils.custom_object_scope({
  'ssim_metric': lambda y_true, y_pred: (y_true, y_pred),
  'psnr_metric': lambda y_true, y_pred: (y_true, y_pred),
  'perceptual_loss': lambda y_true, y_pred: (y_true, y_pred)
}):
  generator = tf.keras.models.load_model(MODEL_PATH)

In [6]:
imloader = ImageLoader()
tqdmloop = tqdm(imloader, desc=f"Evaluating => ")
with open('/content/eval.chk', "w") as evalfile:
  for idx, (hr, lr) in enumerate(tqdmloop):
    sr = generator.predict_on_batch(lr)

    psnr = tf.image.psnr(hr, sr, 1.)
    ssim = tf.image.ssim(hr, sr, 1.)

    evalfile.write(f'{psnr};{ssim}\n')

Evaluating => : 100%|██████████| 100/100 [00:24<00:00,  4.02it/s]
